In [1]:
#import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import dask.dataframe as dd
from IPython.display import display, HTML
import pandas as pd
import cudf

In [2]:
df = cudf.read_parquet('imputed_df_final.parquet')

In [3]:
df.head()

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,customer,merchant
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,0,0,9.140800,107.215862,26.686594,74.000000,3682.750000,138.000000,111.000000,14.0,...,49466.0,65923.0,0.000000,29.180,58.434969,32.500000,86.0,15.856826,168972,152285
1,0,0,13.990614,35.552000,50.928261,3.000000,1171.350000,23.000000,17.000000,2.0,...,3638.0,7801.0,0.419355,28.465,5.392089,7.000000,125.0,6.998555,212404,39032
2,0,0,8.361790,31.623103,48.837872,19.000000,2295.380000,47.000000,42.000000,11.0,...,3912.0,12868.0,0.836364,421.500,33.780445,0.000000,180.0,1.753009,225178,7439
3,0,0,8.098895,112.277391,113.889500,-1.730775,1804.766492,-8.213324,0.689400,16.0,...,28919.0,23553.0,0.952381,50.000,37.340085,28.666667,134.0,9.000063,183948,485069
4,1,0,6.686296,448.427273,164.725085,-1.154068,1298.118567,3.600994,-2.638847,5.0,...,1086.0,308.0,0.754386,69.509,77.794164,15.000000,114.0,1.767939,210107,536004


In [4]:
columns = df.columns.to_list()
columns
remove_list = ['ind_recommended', 'activation', 'customer', 'merchant', 'merchant_profile_01']
for i in remove_list:
    columns.remove(i)

In [5]:
from cuml.preprocessing import MinMaxScaler
import cudf
import cupy as cp

def rescale_column(df, column, scaler):
    """
    Rescales a single column, ensuring compatibility with cuDF DataFrame structure.
    """
    try:
        # Apply fit and transform operations
        scaler.fit(df[[column]])
        scaled_data = scaler.transform(df[[column]])
        # Assuming scaled_data is a DataFrame, select the column as a Series
        if not isinstance(scaled_data, cudf.DataFrame):
            # Assuming scaled_data can be converted to a cuPy array for handling
            scaled_data = cp.asarray(scaled_data).flatten()  # Flatten in case it's not already 1D
            scaled_data = cudf.Series(scaled_data)  # Convert to cuDF Series
            
        else:
            # If it's already a DataFrame, select the column directly
            scaled_data = scaled_data.iloc[:, 0]  # Select the scaled series
            
        return scaled_data
    except Exception as e:
        print(f"Error with column {column}: {e}")
        return df[column]

def rescale_dataset_column_by_column(df, exclude_columns):
    """
    Rescales the dataset column by column, excluding specified columns,
    ensuring unique index values throughout the process.
    """
    scaler = MinMaxScaler()
    
    # Save the original index and reset it to ensure it's unique
    original_index = df.index
    df = df.reset_index(drop=True)
    
    rescaled_df = cudf.DataFrame()
    
    for column in df.columns:
        if column not in exclude_columns:
            rescaled_series = rescale_column(df, column, scaler)
            rescaled_df[column] = rescaled_series
            print(f'{column} done scaling')
        else:
            # Directly assign excluded columns without scaling
            rescaled_df[column] = df[column]
    
    # Restore the original index to the rescaled DataFrame
    rescaled_df.index = original_index
    
    return rescaled_df



df = rescale_dataset_column_by_column(df, remove_list)
df = df.to_pandas()

customer_digital_activity_04 done scaling
customer_spend_01 done scaling
customer_industry_spend_01 done scaling
customer_industry_spend_02 done scaling
customer_industry_spend_03 done scaling
customer_industry_spend_04 done scaling
customer_industry_spend_05 done scaling
customer_spend_02 done scaling
customer_spend_03 done scaling
customer_spend_04 done scaling
customer_spend_05 done scaling
customer_spend_06 done scaling
customer_spend_07 done scaling
merchant_spend_01 done scaling
merchant_spend_02 done scaling
merchant_spend_03 done scaling
merchant_spend_04 done scaling
merchant_spend_05 done scaling
merchant_spend_06 done scaling
merchant_spend_07 done scaling
merchant_spend_08 done scaling
customer_merchant_03 done scaling
customer_profile_01 done scaling
customer_profile_02 done scaling
customer_digital_activity_05 done scaling
customer_spend_13 done scaling
customer_digital_activity_06 done scaling
customer_spend_14 done scaling
customer_digital_activity_10 done scaling
custo

In [6]:
def write_dataframe_to_csv_in_chunks(df, output_file_path, chunksize):
    """
    Writes a cuDF DataFrame to a CSV file in chunks to manage memory usage.

    Parameters:
        df (cudf.DataFrame): The DataFrame to write.
        output_file_path (str): The path to the output CSV file.
        chunksize (int): The number of rows per chunk.
    """
    # Determine the total number of chunks needed
    total_chunks = len(df) // chunksize + (1 if len(df) % chunksize > 0 else 0)
    
    for i in range(total_chunks):
        start_row = i * chunksize
        end_row = start_row + chunksize
        df_chunk = df.iloc[start_row:end_row]
        
        # Determine write mode, write header only for the first chunk
        mode = 'w' if i == 0 else 'a'
        header = True if i == 0 else False
        
        # Write the current chunk to the CSV file
        df_chunk.to_csv(output_file_path, mode=mode, header=header, index=False)
        
        print(f'Finished writing chunk {i+1}/{total_chunks}')

write_dataframe_to_csv_in_chunks(df, 'rescaled_data.csv', chunksize=100000) 

Finished writing chunk 1/123
Finished writing chunk 2/123
Finished writing chunk 3/123
Finished writing chunk 4/123
Finished writing chunk 5/123
Finished writing chunk 6/123
Finished writing chunk 7/123
Finished writing chunk 8/123
Finished writing chunk 9/123
Finished writing chunk 10/123
Finished writing chunk 11/123
Finished writing chunk 12/123
Finished writing chunk 13/123
Finished writing chunk 14/123
Finished writing chunk 15/123
Finished writing chunk 16/123
Finished writing chunk 17/123
Finished writing chunk 18/123
Finished writing chunk 19/123
Finished writing chunk 20/123
Finished writing chunk 21/123
Finished writing chunk 22/123
Finished writing chunk 23/123
Finished writing chunk 24/123
Finished writing chunk 25/123
Finished writing chunk 26/123
Finished writing chunk 27/123
Finished writing chunk 28/123
Finished writing chunk 29/123
Finished writing chunk 30/123
Finished writing chunk 31/123
Finished writing chunk 32/123
Finished writing chunk 33/123
Finished writing ch